In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install geopandas
!pip install plotly 

     |████████████████████████████████| 1.0 MB 6.9 MB/s 
     |████████████████████████████████| 16.7 MB 43.5 MB/s 
     |████████████████████████████████| 6.3 MB 47.3 MB/s 


In [4]:
import geopandas  as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

###2022 dataset

In [5]:
shapefile_path = r"/content/drive/MyDrive/Task2/list/IBTrACS.ACTIVE.list.v04r00.lines.shp"
df1 = gpd.read_file(shapefile_path)

In [6]:
df1.head()

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,USA_SEA_SW,USA_SEA_NW,STORM_SPD,STORM_DR,year,month,day,hour,min,geometry
0,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 12:00:00,NR,-13.10,113.80,...,None,None,13,259,2022,3,20,12,0,"LINESTRING (113.80005 -13.10000, 113.16235 -13..."
1,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 15:00:00,NR,-13.22,113.16,...,None,None,12,251,2022,3,20,15,0,"LINESTRING (113.16235 -13.22043, 112.59998 -13..."
2,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 18:00:00,NR,-13.50,112.60,...,None,None,13,230,2022,3,20,18,0,"LINESTRING (112.59998 -13.50000, 112.16272 -14..."
3,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 21:00:00,NR,-14.03,112.16,...,None,None,14,215,2022,3,20,21,0,"LINESTRING (112.16272 -14.02730, 111.80005 -14..."
4,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-21 00:00:00,NR,-14.60,111.80,...,None,None,12,217,2022,3,21,0,0,"LINESTRING (111.80005 -14.60000, 111.42725 -14..."


In [20]:
df1 = df1.set_geometry('geometry')

In [7]:
df1.columns

Index(['SID', 'SEASON', 'NUMBER', 'BASIN', 'SUBBASIN', 'NAME', 'ISO_TIME',
       'NATURE', 'LAT', 'LON',
       ...
       'USA_SEA_SW', 'USA_SEA_NW', 'STORM_SPD', 'STORM_DR', 'year', 'month',
       'day', 'hour', 'min', 'geometry'],
      dtype='object', length=169)

In [9]:
drop_cols =[]

In [25]:
def handle_null_values(df):
    global drop_cols
    no_null = True
    for col in df.columns:
        null_cnt = df[col].isnull().sum()
        if 0<float(null_cnt)/ float(df.shape[0]) <0.1 and pd.api.types.is_numeric_dtype(df[col]):
            medi = df[col].median()
            df[col]= df[col].apply(lambda x : medi if x==None else x)
            #print(f"Replaced {null_cnt} values in {col} with median {medi}")
            no_null = False
        elif  null_cnt==0:
            no_null = no_null and True
        else:
            drop_cols.append(col)
    if no_null:
        print("No null values found")
    else:
        if len(drop_cols)>0:
            print("Dropped columns : {drop_cols}")
        else:
            print("No columns dropped")
    return (df.drop(columns=drop_cols,inplace= True),drop_cols)

In [28]:
def preprocess(df1):
  handle_null_values(df1)
  #df1.drop(columns=drop_cols,inplace= True)

In [ ]:
preprocess(df1)

In [21]:
df1.head()

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,BOM_TNUM,BOM_CI,STORM_SPD,STORM_DR,year,month,day,hour,min,geometry
0,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 12:00:00,NR,-13.10,113.80,...,-999.9,-999.9,13,259,2022,3,20,12,0,"LINESTRING (113.80005 -13.10000, 113.16235 -13..."
1,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 15:00:00,NR,-13.22,113.16,...,-999.9,-999.9,12,251,2022,3,20,15,0,"LINESTRING (113.16235 -13.22043, 112.59998 -13..."
2,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 18:00:00,NR,-13.50,112.60,...,-999.9,-999.9,13,230,2022,3,20,18,0,"LINESTRING (112.59998 -13.50000, 112.16272 -14..."
3,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 21:00:00,NR,-14.03,112.16,...,-999.9,-999.9,14,215,2022,3,20,21,0,"LINESTRING (112.16272 -14.02730, 111.80005 -14..."
4,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-21 00:00:00,NR,-14.60,111.80,...,-999.9,-999.9,12,217,2022,3,21,0,0,"LINESTRING (111.80005 -14.60000, 111.42725 -14..."


In [84]:
df1.columns

Index(['SID', 'SEASON', 'NUMBER', 'BASIN', 'SUBBASIN', 'NAME', 'ISO_TIME',
       'NATURE', 'LAT', 'LON', 'TRACK_TYPE', 'DIST2LAND', 'LANDFALL', 'IFLAG',
       'USA_ATCFID', 'USA_LAT', 'USA_LON', 'USA_WIND', 'USA_PRES', 'USA_SSHS',
       'USA_POCI', 'USA_ROCI', 'USA_RMW', 'NEW_CI', 'REU_TNUM', 'REU_CI',
       'BOM_TNUM', 'BOM_CI', 'STORM_SPD', 'STORM_DR', 'year', 'month', 'day',
       'hour', 'min', 'geometry'],
      dtype='object')

###last 3 years dataset

In [23]:
shapefile_path = r"/content/drive/MyDrive/Task2/last3years/IBTrACS.last3years.list.v04r00.lines.shp"
df2 = gpd.read_file(shapefile_path)

In [26]:
preprocess(df2)

Dropped columns : {drop_cols}


In [27]:
df2.columns

Index(['SID', 'SEASON', 'NUMBER', 'BASIN', 'SUBBASIN', 'NAME', 'ISO_TIME',
       'NATURE', 'LAT', 'LON',
       ...
       'USA_SEA_SW', 'USA_SEA_NW', 'STORM_SPD', 'STORM_DR', 'year', 'month',
       'day', 'hour', 'min', 'geometry'],
      dtype='object', length=169)

###last 40 year dataset

In [62]:
shapefile_path = r"/content/drive/MyDrive/Task2/since1980/IBTrACS.since1980.list.v04r00.lines.shp"
df3 = gpd.read_file(shapefile_path)

In [124]:
preprocess(df3)

No null values found


In [68]:
df3.columns

Index(['SID', 'SEASON', 'NUMBER', 'BASIN', 'SUBBASIN', 'NAME', 'ISO_TIME',
       'NATURE', 'LAT', 'LON', 'TRACK_TYPE', 'DIST2LAND', 'LANDFALL', 'IFLAG',
       'USA_ATCFID', 'USA_LAT', 'USA_LON', 'USA_WIND', 'USA_PRES', 'USA_SSHS',
       'USA_POCI', 'USA_ROCI', 'USA_RMW', 'NEW_CI', 'REU_TNUM', 'REU_CI',
       'BOM_TNUM', 'BOM_CI', 'STORM_SPD', 'STORM_DR', 'year', 'month', 'day',
       'hour', 'min', 'geometry'],
      dtype='object')

Store the CSV

In [125]:
df1.to_csv(r"/content/drive/MyDrive/Task2/list.csv")

In [126]:
df2.to_csv(r"/content/drive/MyDrive/Task2/last3.csv")

In [127]:
df3.to_csv(r"/content/drive/MyDrive/Task2/last40.csv")

#ETL Operation

In [80]:
import glob 
import pandas as pd 
import xml.etree.ElementTree as ET 
from datetime import datetime

In [81]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe

In [101]:
def extract():
      d=pd.DataFrame()
      for csvfile in glob.glob("/content/drive/MyDrive/Task2/*.csv"):
          d = d.append(extract_from_csv(csvfile), ignore_index=True)
      return d

In [29]:
def transform(df):
      df["Datetime"] = df['year'].astype(str) + '-' + df['month'].astype(str)+'-'+df['day'].astype(str)+' '+df['hour'].astype(str)+'-'+df['min'].astype(str)
      df['Datetime'] = pd.to_datetime(df['Datetime'], format="%Y-%m-%d %H-%M")
      df.drop(columns = ['month','day','year','hour','min'],inplace = True)
      return df

In [85]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

In [128]:
extracted_data = extract()

In [129]:
for csvfile in glob.glob("/content/drive/MyDrive/Task2/*.csv"):
  print(csvfile)

/content/drive/MyDrive/Task2/list.csv
/content/drive/MyDrive/Task2/last3.csv
/content/drive/MyDrive/Task2/last40.csv


In [130]:
transformed_data = transform(extracted_data)

In [131]:
transformed_data

,Unnamed: 0,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,...,BOM_CI,STORM_SPD,STORM_DR,year,month,day,hour,min,geometry,Datetime
0,0,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 12:00:00,NR,-13.10,...,-999.9,13,259,2022,3,20,12,0,LINESTRING (113.800048828125 -13.1000013351440...,2022-03-20 12:00:00
1,1,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 15:00:00,NR,-13.22,...,-999.9,12,251,2022,3,20,15,0,LINESTRING (113.162353515625 -13.2204332351684...,2022-03-20 15:00:00
2,2,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 18:00:00,NR,-13.50,...,-999.9,13,230,2022,3,20,18,0,LINESTRING (112.5999755859375 -13.499999046325...,2022-03-20 18:00:00
3,3,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-20 21:00:00,NR,-14.03,...,-999.9,14,215,2022,3,20,21,0,LINESTRING (112.1627197265625 -14.027301788330...,2022-03-20 21:00:00
4,4,2022080S13114,2022,18,SI,WA,CHARLOTTE,2022-03-21 00:00:00,NR,-14.60,...,-999.9,12,217,2022,3,21,0,0,LINESTRING (111.800048828125 -14.6000003814697...,2022-03-21 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294280,275786,2022083S13076,2022,19,SI,MM,HALIMA,2022-03-30 15:00:00,NR,-19.61,...,-999.9,3,44,2022,3,30,15,0,"LINESTRING (82.35498046875 -19.60733985900879,...",2022-03-30 15:00:00
294281,275787,2022083S13076,2022,19,SI,MM,HALIMA,2022-03-30 18:00:00,NR,-19.50,...,-999.9,5,65,2022,3,30,18,0,"LINESTRING (82.5 -19.5, 82.8402099609375 -19.3...",2022-03-30 18:00:00
294282,275788,2022083S13076,2022,19,SI,MM,HALIMA,2022-03-30 21:00:00,NR,-19.40,...,-999.9,8,75,2022,3,30,21,0,LINESTRING (82.8402099609375 -19.3978767395019...,2022-03-30 21:00:00
294283,275789,2022090N12109,2022,20,WP,MM,NOT_NAMED,2022-03-30 18:00:00,NR,12.30,...,-999.9,5,2,2022,3,30,18,0,LINESTRING (109.199951171875 12.30000114440918...,2022-03-30 18:00:00


In [132]:
load("/content/drive/MyDrive/Task2/final.csv",transformed_data)

#Computing summaries and stats 